In [3]:
import os 
import sys 
import pandas as pd 
import re
import pickle
import json
import sqlite3
from pprint import pprint
import datetime
import pdb

In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

In [7]:
with open('../credentials/spotify_creds.pkl', 'rb') as hnd:
    credentials = pickle.load(hnd)

In [8]:
os.environ.update(credentials)

In [9]:
scope = "user-read-recently-played"

In [10]:
spotify = spotipy.Spotify(client_credentials_manager= SpotifyOAuth(scope=scope
                                                                   , username='malchemist02'))

In [11]:
db_location = 'data/listening_history.db'

In [12]:
con = sqlite3.connect(db_location)
db = pd.read_sql('select distinct artist_id from Listening_History', con)
db

,artist_id
0,spotify:artist:2M72oi0t6i7mADfvCtekOM
1,spotify:artist:5aAlzehdUM14I4ppq24Xob
2,spotify:artist:50JJSqHUf2RQ9xsHs0KMHg
3,spotify:artist:6jTnHxhb6cDCaCu4rdvsQ0
4,spotify:artist:3ApUX1o6oSz321MMECyIYd
...,...
883,spotify:artist:7H6dkUChT5EoOQtUVMg4cN
884,spotify:artist:25NQNriVT2YbSW80ILRWJa
885,spotify:artist:57ALvbCBaCkNlgTOSiUPdT
886,spotify:artist:6xvpfMjWTougrRRtK7iikz


In [14]:
artists_all = db['artist_id'].values.tolist()
artists_all = list(set(artists_all))
print("Number of artists: ", len(artists_all))
artists_all[:100]

Number of artists:  888


['spotify:artist:4TjR4K8rD5CqqqU59V6aGr',
 'spotify:artist:5dCuFngSPyOOnTAvrC7v2s',
 'spotify:artist:2lolQgalUvZDfp5vvVtTYV',
 'spotify:artist:7KHQppBj7ea6qRp4NsTN1u',
 'spotify:artist:3K2Srho6NCF3o9MswGR76H',
 'spotify:artist:6LuN9FCkKOj5PcnpouEgny',
 'spotify:artist:738wLrAtLtCtFOLvQBXOXp',
 'spotify:artist:7v4imS0moSyGdXyLgVTIV7',
 'spotify:artist:5d1JhBfyb58upMXCZOdbQu',
 'spotify:artist:0HSRJAuF3aNdrbivhOb9SC',
 'spotify:artist:3g7vWifmj7qRmNwAR7K1Fc',
 'spotify:artist:4vhNDa5ycK0ST968ek7kRr',
 'spotify:artist:67nwj3Y5sZQLl72VNUHEYE',
 'spotify:artist:3jakCpqlxelvHkdvhPWl2p',
 'spotify:artist:6VDdCwrBM4qQaGxoAyxyJC',
 'spotify:artist:1vyhD5VmyZ7KMfW5gqLgo5',
 'spotify:artist:6sHCvZe1PHrOAuYlwTLNH4',
 'spotify:artist:1LZEQNv7sE11VDY3SdxQeN',
 'spotify:artist:1Yq9Q4bPuxh1N4wtIEXyII',
 'spotify:artist:3cjEqqelV9zb4BYE3qDQ4O',
 'spotify:artist:6vv8DZKpEVRMWVPCy2u6gc',
 'spotify:artist:4MCBfE4596Uoi2O4DtmEMz',
 'spotify:artist:60R5qTUzLPi2jt3OCO4oHw',
 'spotify:artist:1gUi2utSbJLNPddYE

In [ ]:
# artists_rslt = spotify.artists(artists_all[:49])
# artists_rslt

In [18]:
all_rslts = []
for x in range(1+int(len(artists_all)/50)):
    start = x*50
    end = (x+1)*50
    print("Start Num: ", start, " End: ", end, "Started at ", datetime.datetime.now())
    temp_rslts = spotify.artists(artists_all[start:end])
    all_rslts.append(temp_rslts)

Start Num:  0  End:  50 Started at  2020-12-15 19:41:07.497640
Start Num:  50  End:  100 Started at  2020-12-15 19:41:07.599079
Start Num:  100  End:  150 Started at  2020-12-15 19:41:07.655409
Start Num:  150  End:  200 Started at  2020-12-15 19:41:07.731209
Start Num:  200  End:  250 Started at  2020-12-15 19:41:07.782373
Start Num:  250  End:  300 Started at  2020-12-15 19:41:07.867643
Start Num:  300  End:  350 Started at  2020-12-15 19:41:07.917335
Start Num:  350  End:  400 Started at  2020-12-15 19:41:07.996421
Start Num:  400  End:  450 Started at  2020-12-15 19:41:08.054802
Start Num:  450  End:  500 Started at  2020-12-15 19:41:08.108110
Start Num:  500  End:  550 Started at  2020-12-15 19:41:08.182300
Start Num:  550  End:  600 Started at  2020-12-15 19:41:08.286460
Start Num:  600  End:  650 Started at  2020-12-15 19:41:08.347125
Start Num:  650  End:  700 Started at  2020-12-15 19:41:08.403959
Start Num:  700  End:  750 Started at  2020-12-15 19:41:08.467839
Start Num:  75

In [23]:
all_artist_info = []
for x in all_rslts:
    all_artist_info.extend(x['artists'])
print(len(all_artist_info))

50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
38
888


In [24]:
def parse_artist_rslt(artist_dict):
    out = {}
    out['artist_name'] = artist_dict.get('name')
    out['artist_id'] = artist_dict.get('id')
    out['followers'] = artist_dict.get('followers').get('total')
    out['genres'] = artist_dict.get('genres')
    out['popularity'] = artist_dict.get('popularity')
    out['pull_date'] = datetime.datetime.now().date()
    out_series = pd.Series(out)
    return(out_series)

In [33]:
artist_series = [parse_artist_rslt(x) for x in all_artist_info]
artist_df = pd.concat(artist_series, axis=1).T
artist_df['pull_date'] = artist_df['pull_date'].astype('str')
artist_df['genres'] = artist_df['genres'].astype('str')
artist_df.head()

,artist_name,artist_id,followers,genres,popularity,pull_date
0,John Oates,4TjR4K8rD5CqqqU59V6aGr,37165,[],39,2020-12-15
1,Freddie King,5dCuFngSPyOOnTAvrC7v2s,267400,"['blues', 'blues rock', 'classic rock', 'elect...",53,2020-12-15
2,Tony Bennett,2lolQgalUvZDfp5vvVtTYV,589356,"['adult standards', 'easy listening', 'vocal j...",73,2020-12-15
3,Khanipova,7KHQppBj7ea6qRp4NsTN1u,139,['background piano'],47,2020-12-15
4,Bakar,3K2Srho6NCF3o9MswGR76H,134840,['uk alternative hip hop'],66,2020-12-15


In [34]:
artist_df.to_sql('Artists_Info', con, index=False) #, if_exists='replace')

In [36]:
artist_df.sort_values('popularity', ascending=False).head(20)

,artist_name,artist_id,followers,genres,popularity,pull_date
622,Bad Bunny,4q3ewBCX7sLwd24euuV69X,27597359,"['latin', 'reggaeton', 'trap latino']",100,2020-12-15
21,Juice WRLD,4MCBfE4596Uoi2O4DtmEMz,14051748,"['chicago rap', 'melodic rap']",97,2020-12-15
265,Ariana Grande,66CXWjxzNUsdJxJ2JdwvnR,54194762,"['pop', 'post-teen pop']",97,2020-12-15
390,BTS,3Nrfpe0tUJi4K4DXYWgMUX,26200003,"['k-pop', 'k-pop boy group']",97,2020-12-15
105,Drake,3TVXtAsR1Inumwj472S9r4,51321643,"['canadian hip hop', 'canadian pop', 'hip hop'...",96,2020-12-15
268,Taylor Swift,06HL4z0CvFAxyc27GXpf02,34466348,"['dance pop', 'pop', 'pop dance']",96,2020-12-15
269,Justin Bieber,1uNFoZAHBGtllmzznpCI3s,40307619,"['canadian pop', 'pop', 'post-teen pop']",95,2020-12-15
15,J Balvin,1vyhD5VmyZ7KMfW5gqLgo5,24575749,"['latin', 'reggaeton', 'reggaeton colombiano']",95,2020-12-15
777,The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,26634329,"['canadian contemporary r&b', 'canadian pop', ...",94,2020-12-15
515,Pop Smoke,0eDvMgVFoNV3TpwtrVCoTj,2983056,['brooklyn drill'],93,2020-12-15
